In [6]:
#!pip3 install twilio

# Importar librerias

In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time 

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

from datetime import datetime 



# Armado de la URL

In [2]:
query = 'Málaga'
api_key = API_KEY_WAPI

url_clima = f'http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={query}&days=1&aqi=no&alerts=no'

response = requests.get(url_clima).json() 

In [4]:
response

{'location': {'name': 'Málaga',
  'region': 'Andalucia',
  'country': 'Espagne',
  'lat': 36.7167,
  'lon': -4.4167,
  'tz_id': 'Europe/Madrid',
  'localtime_epoch': 1749643249,
  'localtime': '2025-06-11 14:00'},
 'current': {'last_updated_epoch': 1749643200,
  'last_updated': '2025-06-11 14:00',
  'temp_c': 27.1,
  'temp_f': 80.8,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 8.1,
  'wind_kph': 13.0,
  'wind_degree': 166,
  'wind_dir': 'SSE',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 54,
  'cloud': 0,
  'feelslike_c': 27.5,
  'feelslike_f': 81.5,
  'windchill_c': 27.5,
  'windchill_f': 81.4,
  'heatindex_c': 27.9,
  'heatindex_f': 82.2,
  'dewpoint_c': 15.2,
  'dewpoint_f': 59.3,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 9.3,
  'gust_mph': 9.3,
  'gust_kph': 14.9},
 'forecast': {'forecastday': [{'date': '2025-06-11',
    'date

In [5]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [6]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [7]:
response['forecast']['forecastday'][0]['hour']

[{'time_epoch': 1749592800,
  'time': '2025-06-11 00:00',
  'temp_c': 22.8,
  'temp_f': 73.1,
  'is_day': 0,
  'condition': {'text': 'Patchy rain nearby',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
   'code': 1063},
  'wind_mph': 5.8,
  'wind_kph': 9.4,
  'wind_degree': 27,
  'wind_dir': 'NNE',
  'pressure_mb': 1016.0,
  'pressure_in': 29.99,
  'precip_mm': 0.02,
  'precip_in': 0.0,
  'snow_cm': 0.0,
  'humidity': 59,
  'cloud': 87,
  'feelslike_c': 24.8,
  'feelslike_f': 76.7,
  'windchill_c': 22.9,
  'windchill_f': 73.1,
  'heatindex_c': 24.8,
  'heatindex_f': 76.7,
  'dewpoint_c': 14.4,
  'dewpoint_f': 58.0,
  'will_it_rain': 1,
  'chance_of_rain': 89,
  'will_it_snow': 0,
  'chance_of_snow': 0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'gust_mph': 9.7,
  'gust_kph': 15.6,
  'uv': 0},
 {'time_epoch': 1749596400,
  'time': '2025-06-11 01:00',
  'temp_c': 22.2,
  'temp_f': 72.0,
  'is_day': 0,
  'condition': {'text': 'Partly Cloudy ',
   'icon': '//cdn.weatherapi.com

In [10]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0] # Fecha

'2025-06-11'

In [16]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) # Hora

1

In [18]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']  #Condition

'Patchy rain nearby'

In [19]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] #Temepratura

22.8

In [20]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] # si llueve o no

1

In [21]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] # probabilidad de lluvia

89

# Dataframe

In [24]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain =response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [27]:
datos = []

# tqdm permite visualizar una barra de carga en los ciclos for
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'): 
    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [29]:
datos[0]

('2025-06-11', 0, 'Patchy rain nearby', 22.8, 1, 89)

In [50]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia', 'prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2025-06-11,0,Patchy rain nearby,22.8,1,89
1,2025-06-11,1,Partly Cloudy,22.2,0,0
2,2025-06-11,2,Patchy rain nearby,21.9,1,100
3,2025-06-11,3,Patchy rain nearby,23.0,1,86
4,2025-06-11,4,Partly Cloudy,22.8,0,0
5,2025-06-11,5,Patchy rain nearby,21.7,1,76
6,2025-06-11,6,Patchy rain nearby,21.8,0,66
7,2025-06-11,7,Patchy rain nearby,21.9,1,100
8,2025-06-11,8,Moderate rain at times,23.1,1,100
9,2025-06-11,9,Patchy rain nearby,23.9,1,100


In [40]:
# filtrar cuando llueve
df_rain = df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora'] < 22)]
df_rain = df_rain[['Hora', 'Condicion']]
df_rain.set_index('Hora', inplace = True)

In [41]:
df_rain

,Condicion
Hora,
7,Patchy rain nearby
8,Moderate rain at times
9,Patchy rain nearby
10,Patchy rain nearby
11,Patchy rain nearby


# Mensaje - Template

In [88]:
template = f'{str(df_rain)}'
template

'                   Condicion\nHora                        \n7         Patchy rain nearby\n8     Moderate rain at times\n9         Patchy rain nearby\n10        Patchy rain nearby\n11        Patchy rain nearby'

In [60]:
PHONE_NUMBER

'+19129001518'

# Mensaje SMS desde twilio

In [89]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages.create(
    body=template,
    from_=PHONE_NUMBER,
    to="+34744786798",
)

print(f'Mensaje Enviado {message.sid}')

Mensaje Enviado SMa14f5441daba0938ab7a72020ab433eb
